In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm
from pandas.tseries.offsets import BMonthEnd

# 加载数据
data = pd.read_csv('/home/u2024210271/jupyterlab/VIX计算/中证option_daily_prices.csv')
data.columns = data.columns.str.strip()
print("清理后列名:", data.columns.tolist())

# 检查数据是否为空
if data.empty:
    print("错误：数据文件为空，请检查option_daily_prices.csv是否正确加载。")
    exit()

# 打印code列的唯一值以调试
print("code列的唯一值:", data['code'].unique())

# 定义解析code函数
def parse_code(code):
    try:
        # 去掉.CCFX后缀
        code = code.replace('.CCFX', '')
        parts = code.split('-')
        if len(parts) != 3:
            return pd.NA, pd.NA, pd.NA
        base, type_str, strike_str = parts
        if len(base) < 4 or not base[-4:].isdigit():
            return pd.NA, pd.NA, pd.NA
        yymm = base[-4:]
        yy = int(yymm[:2])
        mm = int(yymm[2:])
        year = 2000 + yy
        # 假设到期日为该月第三个周五
        first_day = pd.to_datetime(f'{year}-{mm:02d}-01')
        days_to_first_friday = (4 - first_day.weekday()) % 7
        first_friday = first_day + pd.Timedelta(days=days_to_first_friday)
        third_friday = first_friday + pd.Timedelta(days=14)
        expire_date = third_friday
        strike = float(strike_str)
        option_type = 'call' if type_str == 'C' else 'put' if type_str == 'P' else pd.NA
        return expire_date, strike, option_type
    except Exception as e:
        print(f"解析code '{code}' 时出错: {e}")
        return pd.NA, pd.NA, pd.NA

# 解析code列
parsed = data['code'].apply(lambda x: pd.Series(parse_code(x), index=['expire_datetime', 'strike_price', 'option_type']))
data = data.join(parsed)

# 打印解析后的数据行数
print(f"解析后数据行数: {len(data)}")

# 删除解析失败的行
data = data.dropna(subset=['expire_datetime', 'strike_price', 'option_type'])

# 打印过滤后的数据行数
print(f"过滤后数据行数: {len(data)}")

if data.empty:
    print("错误：数据为空，请检查数据文件是否包含有效期权数据（code列格式应如 'IO2002-C-3600.CCFX'）。")
    exit()

# 设置date和option_price
data['date'] = pd.to_datetime(data['date'], errors='coerce')
data['option_price'] = data['settle_price']

# 删除关键列中含有NA的行
data = data.dropna(subset=['date', 'expire_datetime', 'strike_price', 'option_price', 'option_type'])

# 打印最终数据行数
print(f"最终数据行数: {len(data)}")

if data.empty:
    print("错误：数据为空，请检查date和settle_price列是否包含有效值。")
    exit()

# 计算到期时间（年）
data['maturity'] = (data['expire_datetime'] - data['date']) / pd.Timedelta(days=365.25)

# 获取交易日
trading_days = sorted(data['date'].unique())

# 存储VIX结果
vix_results = []

# 计算方差函数
def compute_variance(data, exp, datetime, maturity):
    exp_data = data[data['expire_datetime'] == exp]
    pivot = exp_data.pivot_table(index='strike_price', columns='option_type', values='option_price', aggfunc='mean')
    pivot = pivot.fillna(0)

    if len(pivot) < 1:
        print(f"到期日 {exp} 无有效执行价数据。")
        return None, None

    if 'call' not in pivot.columns or 'put' not in pivot.columns:
        print(f"到期日 {exp} 缺少看涨或看跌期权数据。")
        return None, None

    # 计算远期价格
    pivot['diff'] = abs(pivot['call'] - pivot['put'])
    min_diff_strike = pivot['diff'].idxmin()
    C = pivot.loc[min_diff_strike, 'call']
    P = pivot.loc[min_diff_strike, 'put']
    F = min_diff_strike + (C - P)

    # 找到K0
    strikes = pivot.index
    K0_candidates = [K for K in strikes if K <= F]
    K0 = max(K0_candidates) if K0_candidates else min(strikes)

    # 选择期权
    selected = []
    for K in strikes:
        if K < K0 and 'put' in pivot.columns and pivot.loc[K, 'put'] > 0:
            selected.append((K, 'put', pivot.loc[K, 'put']))
        elif K > K0 and 'call' in pivot.columns and pivot.loc[K, 'call'] > 0:
            selected.append((K, 'call', pivot.loc[K, 'call']))
        elif K == K0:
            call_price = pivot.loc[K, 'call'] if 'call' in pivot.columns else 0
            put_price = pivot.loc[K, 'put'] if 'put' in pivot.columns else 0
            average = (call_price + put_price) / 2 if call_price > 0 and put_price > 0 else call_price or put_price
            if average > 0:
                selected.append((K, 'average', average))

    if len(selected) == 0:
        print(f"到期日 {exp} 没有选中的期权。")
        return None, None
    elif len(selected) == 1:
        K, opt_type, price = selected[0]
        ΔK = 25
        sum_contrib = (ΔK / K**2) * price
        print(f"到期日 {exp} 只有1个期权，使用默认ΔK=25")
    else:
        selected.sort(key=lambda x: x[0])
        contributions = []
        for i, (K, opt_type, price) in enumerate(selected):
            if i == 0:
                ΔK = selected[1][0] - K
            elif i == len(selected) - 1:
                ΔK = K - selected[i-1][0]
            else:
                ΔK = (selected[i+1][0] - selected[i-1][0]) / 2
            if ΔK > 0:
                contributions.append((ΔK / K**2) * price)
        sum_contrib = sum(contributions)

    T = maturity
    if T <= 5 / 365.25:
        print(f"到期日 {exp} 到期时间过短（{T*365.25:.2f}天）。")
        return None, None

    variance = (2 / T) * sum_contrib - (1 / T) * ((F / K0 - 1)**2)
    return variance, T

# 单一到期日VIX计算
def compute_single_term_vix(data, exp, datetime, maturity):
    variance, T = compute_variance(data, exp, datetime, maturity)
    if variance is None or T is None:
        return None
    if variance >= 0:
        VIX = 100 * np.sqrt(variance)
        if not np.isnan(VIX) and 0 <= VIX <= 100:
            return VIX
    return None

# 循环处理交易日
for trading_day in tqdm(trading_days, desc='计算VIX'):
    print(f"处理交易日 {trading_day}")
    current_data = data[data['date'] == trading_day]
    
    expirations = current_data[(current_data['expire_datetime'] > trading_day) & 
                              (current_data['maturity'] >= 5 / 365.25)]['expire_datetime'].unique()
    
    print(f"  可用到期日: {len(expirations)}")
    if len(expirations) == 0:
        print(f"交易日 {trading_day} 无可用到期日数据。")
        continue
    
    # 双到期日计算
    if len(expirations) >= 2:
        days_to_expiration = [(exp - trading_day).total_seconds() / 86400 for exp in expirations]
        sorted_expirations = sorted(zip(days_to_expiration, expirations))
        target_day = 30
        min_diff = float('inf')
        near_term_exp, next_term_exp = None, None
        for i in range(len(sorted_expirations) - 1):
            t1, t2 = sorted_expirations[i][0], sorted_expirations[i+1][0]
            diff = abs(t1 - target_day) + abs(t2 - target_day)
            if diff < min_diff:
                near_term_exp = sorted_expirations[i][1]
                next_term_exp = sorted_expirations[i+1][1]
                min_diff = diff
        
        T1 = (near_term_exp - trading_day).total_seconds() / (365.25 * 86400)
        T2 = (next_term_exp - trading_day).total_seconds() / (365.25 * 86400)
        
        σ1_sq, _ = compute_variance(current_data, near_term_exp, trading_day, T1)
        σ2_sq, _ = compute_variance(current_data, next_term_exp, trading_day, T2)
        
        if σ1_sq is not None and σ2_sq is not None:
            τ_30 = 30 / 365.25
            weight_near = (T2 - τ_30) / (T2 - T1)
            weight_next = (τ_30 - T1) / (T2 - T1)
            weight_near = max(0, min(1, weight_near))
            weight_next = max(0, min(1, weight_next))
            σ_sq_30 = weight_near * σ1_sq + weight_next * σ2_sq
            if σ_sq_30 >= 0:
                VIX = 100 * np.sqrt(σ_sq_30)
                if not np.isnan(VIX) and 0 <= VIX <= 100:
                    vix_results.append({'date': trading_day, 'vix': VIX})
                    continue
    
    # 单一到期日计算
    if len(expirations) >= 1:
        days_to_expiration = [(exp - trading_day).total_seconds() / 86400 for exp in expirations]
        sorted_expirations = sorted(zip(days_to_expiration, expirations))
        target_day = 30
        min_diff = float('inf')
        single_exp = None
        for days, exp in sorted_expirations:
            diff = abs(days - target_day)
            if diff < min_diff:
                single_exp = exp
                min_diff = diff
        
        T = (single_exp - trading_day).total_seconds() / (365.25 * 86400)
        VIX = compute_single_term_vix(current_data, single_exp, trading_day, T)
        if VIX is not None:
            vix_results.append({'date': trading_day, 'vix': VIX})
            continue
    
    print(f"交易日 {trading_day} 无法计算VIX。")

# 转换为DataFrame
vix_df = pd.DataFrame(vix_results)

if vix_df.empty:
    print("错误：无法生成VIX数据，请检查数据完整性（确保包含足够看涨和看跌期权数据）。")
    exit()

vix_df = vix_df.sort_values('date')

# 创建完整交易日时间序列
all_trading_days = pd.date_range(start=min(trading_days), end=max(trading_days), freq='B')
vix_df = vix_df.set_index('date').reindex(all_trading_days).reset_index().rename(columns={'index': 'date'})

# 插值和填充缺失值
vix_df['vix'] = vix_df['vix'].interpolate(method='linear').ffill().bfill()

# 绘图
plt.figure(figsize=(12, 6))
plt.plot(vix_df['date'], vix_df['vix'], linestyle='-', color='b')
plt.title('VIX指数随时间变化')
plt.xlabel('日期')
plt.ylabel('VIX值')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('中证vix_time_series.png')
plt.close()

# 保存为Excel文件
vix_df.to_csv('中证vix_results.csv', index=False)

print("VIX计算完成，结果已保存至vix_time_series.png和vix_results.xlsx")

清理后列名: ['id', 'code', 'exchange_code', 'date', 'pre_settle', 'pre_close', 'open', 'high', 'low', 'close', 'change_pct_close', 'settle_price', 'change_pct_settle', 'volume', 'money', 'position']
code列的唯一值: ['MO2208-C-6300.CCFX' 'MO2208-C-6400.CCFX' 'MO2208-C-6500.CCFX' ...
 'MO2507-P-5500.CCFX' 'MO2507-P-6600.CCFX' 'MO2507-C-6600.CCFX']
解析后数据行数: 161548
过滤后数据行数: 161548
最终数据行数: 161548


计算VIX:   2%|▏         | 11/668 [00:00<00:06, 107.73it/s]

处理交易日 2022-07-22 00:00:00
  可用到期日: 6
处理交易日 2022-07-25 00:00:00
  可用到期日: 6
处理交易日 2022-07-26 00:00:00
  可用到期日: 6
处理交易日 2022-07-27 00:00:00
  可用到期日: 6
处理交易日 2022-07-28 00:00:00
  可用到期日: 6
处理交易日 2022-07-29 00:00:00
  可用到期日: 6
处理交易日 2022-08-01 00:00:00
  可用到期日: 6
处理交易日 2022-08-02 00:00:00
  可用到期日: 6
处理交易日 2022-08-03 00:00:00
  可用到期日: 6
处理交易日 2022-08-04 00:00:00
  可用到期日: 6
处理交易日 2022-08-05 00:00:00
  可用到期日: 6
处理交易日 2022-08-08 00:00:00
  可用到期日: 6
处理交易日 2022-08-09 00:00:00
  可用到期日: 6
处理交易日 2022-08-10 00:00:00
  可用到期日: 6
处理交易日 2022-08-11 00:00:00
  可用到期日: 6
处理交易日 2022-08-12 00:00:00
  可用到期日: 6
处理交易日 2022-08-15 00:00:00
  可用到期日: 5
处理交易日 2022-08-16 00:00:00
  可用到期日: 5
处理交易日 2022-08-17 00:00:00
  可用到期日: 5
处理交易日 2022-08-18 00:00:00
  可用到期日: 5
处理交易日 2022-08-19 00:00:00
  可用到期日: 5
处理交易日 2022-08-22 00:00:00
  可用到期日: 6


计算VIX:   5%|▌         | 34/668 [00:00<00:05, 111.06it/s]

处理交易日 2022-08-23 00:00:00
  可用到期日: 6
处理交易日 2022-08-24 00:00:00
  可用到期日: 6
处理交易日 2022-08-25 00:00:00
  可用到期日: 6
处理交易日 2022-08-26 00:00:00
  可用到期日: 6
处理交易日 2022-08-29 00:00:00
  可用到期日: 6
处理交易日 2022-08-30 00:00:00
  可用到期日: 6
处理交易日 2022-08-31 00:00:00
  可用到期日: 6
处理交易日 2022-09-01 00:00:00
  可用到期日: 6
处理交易日 2022-09-02 00:00:00
  可用到期日: 6
处理交易日 2022-09-05 00:00:00
  可用到期日: 6
处理交易日 2022-09-06 00:00:00
  可用到期日: 6
处理交易日 2022-09-07 00:00:00
  可用到期日: 6
处理交易日 2022-09-08 00:00:00
  可用到期日: 6
处理交易日 2022-09-09 00:00:00
  可用到期日: 6
处理交易日 2022-09-13 00:00:00
  可用到期日: 5
处理交易日 2022-09-14 00:00:00
  可用到期日: 5
处理交易日 2022-09-15 00:00:00
  可用到期日: 5
处理交易日 2022-09-16 00:00:00
  可用到期日: 5
处理交易日 2022-09-19 00:00:00
  可用到期日: 6
处理交易日 2022-09-20 00:00:00
  可用到期日: 6
处理交易日 2022-09-21 00:00:00
  可用到期日: 6
处理交易日 2022-09-22 00:00:00
  可用到期日: 6
处理交易日 2022-09-23 00:00:00
  可用到期日: 6


...

处理交易日 2025-03-21 00:00:00
  可用到期日: 5
处理交易日 2025-03-24 00:00:00
  可用到期日: 6
处理交易日 2025-03-25 00:00:00
  可用到期日: 6
处理交易日 2025-03-26 00:00:00
  可用到期日: 6
处理交易日 2025-03-27 00:00:00
  可用到期日: 6
处理交易日 2025-03-28 00:00:00
  可用到期日: 6
处理交易日 2025-03-31 00:00:00
  可用到期日: 6
处理交易日 2025-04-01 00:00:00
  可用到期日: 6
处理交易日 2025-04-02 00:00:00
  可用到期日: 6
处理交易日 2025-04-03 00:00:00
  可用到期日: 6
处理交易日 2025-04-07 00:00:00
  可用到期日: 6
处理交易日 2025-04-08 00:00:00
  可用到期日: 6
处理交易日 2025-04-09 00:00:00
  可用到期日: 6
处理交易日 2025-04-10 00:00:00
  可用到期日: 6
处理交易日 2025-04-11 00:00:00
  可用到期日: 6
处理交易日 2025-04-14 00:00:00
  可用到期日: 5
处理交易日 2025-04-15 00:00:00
  可用到期日: 5
处理交易日 2025-04-16 00:00:00
  可用到期日: 5
处理交易日 2025-04-17 00:00:00
  可用到期日: 5
处理交易日 2025-04-18 00:00:00
  可用到期日: 5
处理交易日 2025-04-21 00:00:00
  可用到期日: 6
处理交易日 2025-04-22 00:00:00
  可用到期日: 6
处理交易日 2025-04-23 00:00:00


计算VIX: 100%|██████████| 668/668 [00:06<00:00, 107.81it/s]
/tmp/ipykernel_45769/2031392135.py:262: UserWarning: Glyph 26085 (\N{CJK UNIFIED IDEOGRAPH-65E5}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_45769/2031392135.py:262: UserWarning: Glyph 26399 (\N{CJK UNIFIED IDEOGRAPH-671F}) missing from current font.
  plt.tight_layout()


  可用到期日: 6
处理交易日 2025-04-24 00:00:00
  可用到期日: 6


/tmp/ipykernel_45769/2031392135.py:262: UserWarning: Glyph 20540 (\N{CJK UNIFIED IDEOGRAPH-503C}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_45769/2031392135.py:262: UserWarning: Glyph 25351 (\N{CJK UNIFIED IDEOGRAPH-6307}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_45769/2031392135.py:262: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_45769/2031392135.py:262: UserWarning: Glyph 38543 (\N{CJK UNIFIED IDEOGRAPH-968F}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_45769/2031392135.py:262: UserWarning: Glyph 26102 (\N{CJK UNIFIED IDEOGRAPH-65F6}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_45769/2031392135.py:262: UserWarning: Glyph 38388 (\N{CJK UNIFIED IDEOGRAPH-95F4}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_45769/2031392135.py:262: UserWarning: Glyph 21464 (\N{CJK UNIFIED IDEOGRAPH-53D8}) missing from current font.

VIX计算完成，结果已保存至vix_time_series.png和vix_results.xlsx
